In [1]:
import tensorflow as tf
import json
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# todo:
- flask
- hero functions to `hero.py`
- Overall win rate dire/radiant
- win rates by faction per hero?
- try data augmentation by reflecting the teams. should increase accuracy?

In [2]:
# refactor to heroes.py
def load_heroes():
    data = json.load(open('heroes.json'))

    ids = list(map(lambda x: x['id'], data))
    names = list(map(lambda x: x['localized_name'], data))
    
    id_to_offset = {} 
    offset_to_id = {}
    for offset, id in enumerate(ids):
        id_to_offset[id] = offset
        offset_to_id[offset] = id
        
    return dict(zip(ids, names)), id_to_offset, offset_to_id

heroes, id_to_offset, offset_to_id = load_heroes()

def name_for_offset(o):
    return heroes[offset_to_id[o]]


In [3]:
ANTIMAGE=1
AXE=2
BANE=3
BLOODSEEKER=4
CRYSTAL_MAIDEN=5
DROW_RANGER=6
EARTHSHAKER=7
JUGGERNAUT=8
MIRANA=9
MORPHLING=10
SHADOW_FIEND=11
PHANTOM_LANCER=12
PUCK=13
PUDGE=14
RAZOR=15
SAND_KING=16
STORM_SPIRIT=17
SVEN=18
TINY=19
VENGEFUL_SPIRIT=20
WINDRANGER=21
ZEUS=22
KUNKKA=23
LINA=25
LION=26
SHADOW_SHAMAN=27
SLARDAR=28
TIDEHUNTER=29
WITCH_DOCTOR=30
LICH=31
RIKI=32
ENIGMA=33
TINKER=34
SNIPER=35
NECROPHOS=36
WARLOCK=37
BEASTMASTER=38
QUEEN_OF_PAIN=39
VENOMANCER=40
FACELESS_VOID=41
WRAITH_KING=42
DEATH_PROPHET=43
PHANTOM_ASSASSIN=44
PUGNA=45
TEMPLAR_ASSASSIN=46
VIPER=47
LUNA=48
DRAGON_KNIGHT=49
DAZZLE=50
CLOCKWERK=51
LESHRAC=52
NATURES_PROPHET=53
LIFESTEALER=54
DARK_SEER=55
CLINKZ=56
OMNIKNIGHT=57
ENCHANTRESS=58
HUSKAR=59
NIGHT_STALKER=60
BROODMOTHER=61
BOUNTY_HUNTER=62
WEAVER=63
JAKIRO=64
BATRIDER=65
CHEN=66
SPECTRE=67
ANCIENT_APPARITION=68
DOOM=69
URSA=70
SPIRIT_BREAKER=71
GYROCOPTER=72
ALCHEMIST=73
INVOKER=74
SILENCER=75
OUTWORLD_DEVOURER=76
LYCAN=77
BREWMASTER=78
SHADOW_DEMON=79
LONE_DRUID=80
CHAOS_KNIGHT=81
MEEPO=82
TREANT_PROTECTOR=83
OGRE_MAGI=84
UNDYING=85
RUBICK=86
DISRUPTOR=87
NYX_ASSASSIN=88
NAGA_SIREN=89
KEEPER_OF_THE_LIGHT=90
IO=91
VISAGE=92
SLARK=93
MEDUSA=94
TROLL_WARLORD=95
CENTAUR_WARRUNNER=96
MAGNUS=97
TIMBERSAW=98
BRISTLEBACK=99
TUSK=100
SKYWRATH_MAGE=101
ABADDON=102
ELDER_TITAN=103
LEGION_COMMANDER=104
TECHIES=105
EMBER_SPIRIT=106
EARTH_SPIRIT=107
UNDERLORD=108
TERRORBLADE=109
PHOENIX=110
ORACLE=111
WINTER_WYVERN=112
ARC_WARDEN=113
MONKEY_KING=114
DARK_WILLOW=119
PANGOLIER=120

In [4]:
def vector_from_teams(radiant, dire):
    assert len(radiant) == 5
    assert len(dire) == 5
    v = np.zeros(len(id_to_offset))
    for player in radiant:
        v[id_to_offset[int(player)]] = 1

    for player in dire:
        v[id_to_offset[int(player)]] = -1

    return np.array([v])

In [9]:
heroes_columns = tf.feature_column.numeric_column('heroes', (115,))
estimator = tf.estimator.DNNClassifier(
    feature_columns=[heroes_columns],
    hidden_units=[100, 50, 10],
    model_dir='models/dnn_dota_serve'
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/dnn_dota_serve', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000016FF8A17470>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [11]:
def pred(model, radiant, dire):
    vec = vector_from_teams(radiant, dire)
    print(vec)
    prediction = model.predict(input_fn=tf.estimator.inputs.numpy_input_fn({'heroes': vec}, shuffle=False))
    return prediction

p = pred(
    estimator,
    [DEATH_PROPHET, TERRORBLADE, ANTIMAGE, CENTAUR_WARRUNNER, NATURES_PROPHET], 
    [PHANTOM_ASSASSIN, WRAITH_KING, LYCAN, TREANT_PROTECTOR, TROLL_WARLORD]
)
print(list(p))

p = pred(
    estimator, 
    [PHANTOM_ASSASSIN, WRAITH_KING, LYCAN, TREANT_PROTECTOR, TROLL_WARLORD],
    [DEATH_PROPHET, TERRORBLADE, ANTIMAGE, CENTAUR_WARRUNNER, NATURES_PROPHET]
)
print(list(p))

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0. -1.  1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0. -1.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0. -1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
   0.  0.  0.  0.  0.  0.  0.]]
INFO:tensorflow:Restoring parameters from models/dnn_dota_serve\model.ckpt-5076
[{'logits': array([-0.49891353], dtype=float32), 'logistic': array([0.377796], dtype=float32), 'probabilities': array([0.62220395, 0.37779602], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}]
[[-1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  1. -1.  1.  0.  0.  0.  0.  0.  0.  0.

In [12]:
from flask import Flask, request

app = Flask(__name__)

@app.route('/match/<string:radiant>/<string:dire>/<string:bans>')
def match(radiant, dire, bans):
    return "teams are radiant: {}, dire: {}, bans: {}".format(radiant, dire, bans)


In [ ]:
pred(
    model,
    [TECHIES, IO, OGRE_MAGI, SHADOW_SHAMAN, SHADOW_FIEND],
    [TROLL_WARLORD, VENGEFUL_SPIRIT, GYROCOPTER, PUDGE, TEMPLAR_ASSASSIN]
)

pred(
    model,
    [TROLL_WARLORD, VENGEFUL_SPIRIT, GYROCOPTER, PUDGE, TEMPLAR_ASSASSIN],
    [TECHIES, IO, OGRE_MAGI, SHADOW_SHAMAN, SHADOW_FIEND]
)


In [ ]:
pred(
    model,
    [TINY, KUNKKA, CENTAUR_WARRUNNER, TECHIES, ZEUS],
    [PUDGE, VENOMANCER, LEGION_COMMANDER, JUGGERNAUT, INVOKER]
)

pred(
    model,
    [PUDGE, VENOMANCER, LEGION_COMMANDER, JUGGERNAUT, INVOKER],
    [TINY, KUNKKA, CENTAUR_WARRUNNER, TECHIES, ZEUS]
)

In [17]:
r = np.random.rand()
print(r)
print(1-10**(- r + 1)) 

0.2721787118692156
-4.3434443179788875
